EXCEL SHEET

In [20]:
import pandas as pd

input_xlsx = "prices3.xlsx"
df = pd.read_excel(input_xlsx)

df["sku"] = df["sku"].astype(str)

df.head()

,sku,special price
0,ORL-00006341,6.600
1,ORL-00006341,6.601
2,I-00000190608,15.000
3,I-00000190608,15.001
4,ORL-00006239,22.500


In [21]:
# Step 4 — ultra-short concurrent run (Jupyter)

import asyncio, json, re, math
from aiohttp import ClientSession, TCPConnector, ClientTimeout
from selectolax.parser import HTMLParser

API_URL  = "https://ksa-api.boutiqaat.com/searchplus/rest/V2/global/suggest"
PAGE_URL = "https://www.boutiqaat.com/en-kw/men/{slug}"

HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "origin": "https://www.boutiqaat.com",
    "referer": "https://www.boutiqaat.com/",
    "user-agent": "Mozilla/5.0"
}
BASE_PAYLOAD = {"productId":"","tvId":"","slug":"","celebrityId":"","categoryId":"","brandId":"",
                "celebrityIds":"","categoryIds":"","brandIds":"","optionId":"","attributeId":"",
                "countryCodeAndLanguage":"kw_en","numberOfRecords":20,"newOrAll":"","featuredOnly":None,
                "sortKey":"","sortDirection":"","searchString":""}



def to_float(x):
    if x is None or (isinstance(x, float) and math.isnan(x)): return None
    m = re.search(r"-?\d+(?:\.\d+)?", str(x).replace(",",""))
    return float(m.group(0)) if m else None

async def one(session, sku, expected):
    p = dict(BASE_PAYLOAD); p["searchString"] = sku
    async with session.post(API_URL, headers=HEADERS, json=p) as r:
        j = await r.json(content_type=None)

    slug = j["data"][0]["data"][0]["slug"]
    async with session.get(PAGE_URL.format(slug=slug)) as r:
        html = await r.text()

    j2 = json.loads(HTMLParser(html).css_first('script#__NEXT_DATA__').text())

    price = to_float(j2["props"]["pageProps"]["response"][0]["final_price_with_tax"])
    
    return {"sku": sku, "slug": slug, "web_price": price, "expected_price": expected,
            "match": (None if (price is None or expected is None) else abs(price-expected) < 1e-6)}

async def run_fast(df, limit=50):
    conn = TCPConnector(limit=limit, ssl=False)
    async with ClientSession(connector=conn, timeout=ClientTimeout(total=25)) as s:
        tasks = [one(s, row["sku"], to_float(row["special price"])) for _, row in df.iterrows()]
        res = await asyncio.gather(*tasks, return_exceptions=True)
    # squash exceptions to simple rows
    out = []
    for r in res:
        if isinstance(r, dict): out.append(r)
        else: out.append({"sku": None, "slug": None, "web_price": None, "expected_price": None, "match": None})
    return pd.DataFrame(out)

# Jupyter: just await it
out = await run_fast(df, limit=60)  # bump limit for more speed if stable
merged = df.merge(out[["sku","web_price","match"]], on="sku", how="left")
display(merged.head())


,sku,special price,web_price,match
0,ORL-00006341,6.600,NaN,NaN
1,ORL-00006341,6.601,NaN,NaN
2,I-00000190608,15.000,NaN,NaN
3,I-00000190608,15.001,NaN,NaN
4,ORL-00006239,22.500,NaN,NaN
